In [0]:
dbutils.widgets.text("File_Name", "")


In [0]:
p_file_name = dbutils.widgets.get("File_Name")

In [0]:
p_file_name

**Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://source@ssdatalakeeteproject.dfs.core.windows.net/orders")
display(df)

In [0]:
df = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format","parquet") \
    .option("cloudFiles.schemaLocation",f"abfss://bronze@ssdatalakeeteproject.dfs.core.windows.net/checkpoint_{p_file_name}") \
    .load(f"abfss://source@ssdatalakeeteproject.dfs.core.windows.net/{p_file_name}")

### **Data Writing**

In [0]:
df.writeStream.format("parquet") \
    .outputMode("append") \
    .option("checkpointLocation",f"abfss://bronze@ssdatalakeeteproject.dfs.core.windows.net/checkpoint_{p_file_name}") \
    .option("path",f"abfss://bronze@ssdatalakeeteproject.dfs.core.windows.net/{p_file_name}") \
    .trigger(once=True) \
    .start()

In [0]:
df = spark.read.format("parquet").load(f"abfss://bronze@ssdatalakeeteproject.dfs.core.windows.net/{p_file_name}")
display(df)